In [2]:
import pandas as pd
# Getting data and processing
dataset = pd.read_csv('final_data.csv')
dataset = pd.get_dummies(dataset, columns=['label'])
dataset = dataset[[
    'red', 
    'green', 
    'blue', 
    'label_Red', 
    'label_Green', 
    'label_Blue', 
    'label_Yellow', 
    'label_Orange', 
    'label_Pink', 
    'label_Purple', 
    'label_Brown', 
    'label_Grey', 
    'label_Black', 
    'label_White'
]]

In [3]:
# Split the data into train and test
train_dataset = dataset.sample(frac=0.8, random_state=9)
test_dataset = dataset.drop(train_dataset.index)

In [4]:
#Split features: `red`, `green`, `blue` and labels
train_labels = pd.DataFrame([train_dataset.pop(x) for x in [
    'label_Red', 
    'label_Green', 
    'label_Blue', 
    'label_Yellow', 
    'label_Orange', 
    'label_Pink', 
    'label_Purple', 
    'label_Brown', 
    'label_Grey', 
    'label_Black', 
    'label_White'
]]).T

test_labels = pd.DataFrame([test_dataset.pop(x) for x in [
    'label_Red', 
    'label_Green', 
    'label_Blue', 
    'label_Yellow', 
    'label_Orange', 
    'label_Pink', 
    'label_Purple', 
    'label_Brown', 
    'label_Grey', 
    'label_Black', 
    'label_White'
]]).T


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Model
model = keras.Sequential([
    layers.Dense(3, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(32, activation='relu'),
    layers.Dense(11)
  ])

optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                128       
_________________________________________________________________
dense_2 (Dense)              (None, 11)                363       
Total params: 503
Trainable params: 503
Non-trainable params: 0
_________________________________________________________________


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
# Train the model
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, min_delta=0.01, patience=100)
history = model.fit(x=train_dataset, y=train_labels, 
                    validation_split=0.2, 
                    epochs=1000, 
                    batch_size=32, 
                    verbose=0,
                    callbacks=[early_stop], 
                    shuffle=True)

model.save('model.h5')

Epoch 00509: early stopping


In [7]:
model.evaluate(x=train_dataset, y=train_labels)

127/127 [==============================] - 0s 3ms/step - loss: 0.3156 - accuracy: 0.8728


[0.3155709207057953, 0.8728352189064026]

In [8]:
model.evaluate(x=test_dataset, y=test_labels)

32/32 [==============================] - 0s 1ms/step - loss: 0.3315 - accuracy: 0.8802


[0.33153748512268066, 0.8801980018615723]